In [1]:
from helper import *
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import shelve
import joblib
import random

%matplotlib inline

with shelve.open('shelved_data/IAM_Data') as shelf:
    vocabulary = shelf['chars']
    list_of_images = shelf['list_of_images']
    image_labels = shelf['image_labels']
    
image_arrays = joblib.load('shelved_data/image_arrays')


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


FileNotFoundError: [Errno 2] No such file or directory: 'shelved_data/IAM_Data.dat'

In [ ]:
#List_images ko sort karo
list_of_images.sort()

#Convert vocabulary to list
vocabulary = list(vocabulary)
#Sort so as to have the same ordering every time..
vocabulary.sort()
vocabulary.append("<Blank>")

#Model parameters
hidden_units = 200
num_of_features = 310
vocab_size = len(vocabulary)

#batch_size = 5

#Hyper Parameters
alpha = 0.00001
epochs = 50

losses = []

# first_image = list_of_images[0]
# label  = image_labels[first_image]
# nparray = image_arrays[first_image]
# one_hot_target = one_hot(label,vocabulary)

In [ ]:
#encode_int(label,vocabulary)

In [ ]:
def sparse_tuple_from(sequences, dtype=np.int32):
    """Create a sparse representention of x.
    Args:
        sequences: a list of lists of type dtype where each element is a sequence
    Returns:
        A tuple with (indices, values, shape)
    """
    indices = []
    values = []

    for n, seq in enumerate(sequences):
        indices.extend(zip([n] * len(seq), range(len(seq))))
        values.extend(seq)

    indices = np.asarray(indices, dtype=np.int64)
    values = np.asarray(values, dtype=dtype)
    shape = np.asarray([len(sequences), np.asarray(indices).max(0)[1] + 1], dtype=np.int64)

    return indices, values, shape

# RNN Model

In [ ]:


# Inputs and labels
X = tf.placeholder(tf.float32, [None, None, num_of_features])

targets = tf.sparse_placeholder(tf.int32)

seq_len = tf.placeholder(tf.int32,[None])


# RNN Cell
cell_hidden = tf.contrib.rnn.LSTMCell(hidden_units,num_proj = vocab_size)
outputs,states = tf.nn.dynamic_rnn(cell_hidden,X,seq_len,dtype=tf.float32)


# shape = tf.shape(X)
# batch_s , max_time_steps = shape[0], shape[1]

# # Reshaping to apply the same weights over the timesteps
# outputs = tf.reshape(outputs, [-1, hidden_units])

# # Truncated normal with mean 0 and stdev=0.1
# # Tip: Try another initialization
# # see https://www.tensorflow.org/versions/r0.9/api_docs/python/contrib.layers.html#initializers
# W = tf.Variable(tf.truncated_normal([hidden_units,
#                                      vocab_size],
#                                     stddev=0.1))
# # Zero initialization
# # Tip: Is tf.zeros_initializer the same?
# b = tf.Variable(tf.constant(0., shape=[vocab_size]))

# # Doing the affine projection
# logits = tf.matmul(outputs, W) + b

# # Reshaping back to the original shape
# logits = tf.reshape(logits, [batch_s, -1, vocab_size])

# Time major
logits = tf.transpose(outputs, (1, 0, 2))

#Calculate loss
loss = tf.nn.ctc_loss(targets, logits, seq_len)
cost = tf.reduce_mean(loss)

#Optimize
optimizer = tf.train.AdamOptimizer(learning_rate=alpha)
train = optimizer.minimize(loss)

# CTC decoder.
decoded, log_prob = tf.nn.ctc_greedy_decoder(logits, seq_len)

label_error_rate = tf.reduce_mean(tf.edit_distance(tf.cast(decoded[0], tf.int32),
                                                   targets))


## Save my MoDel

In [ ]:
losses = [] 
outputs = []

saver = tf.train.Saver()

#Take first 52 images
training_list = list_of_images[:52]

random.seed(100)
random.shuffle(training_list)

batches_x = [image_arrays[training_list[i]].T / 255.0 for i in range(len(training_list))]

batches_y = [     
                sparse_tuple_from ([
                                                encode_int(
                                                            image_labels[training_list[i-1]],vocabulary
                                                       )
                    
                        
                                    ] ) for i in range(len(training_list))
    
            ]

    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for e in range(epochs): 

        #Iterate through all images in a single epoch...
        for b in range(len(batches_x)):
                
            #Before feeding x reshape it as (1,original_dims...), i.e make it 3D
                       
            feed = {X:batches_x[b].reshape(1,batches_x[b].shape[0],batches_x[b].shape[1]),targets:batches_y[b],seq_len:[batches_x[b].shape[0]]}

            sess.run(train,feed_dict=feed)

        if e % 1 == 0:
            cost_val,ler_val = sess.run([cost,label_error_rate], feed_dict=feed)
            print("Epoch {0}: cost = {1} ler = {2}".format(e,cost_val,ler_val))
            losses.append(cost)

        #d = sess.run(decoded[0], feed_dict=feed)
        #outputs.append(d)
            
    saver.save(sess,'./Word_RNN.ckpt')

plt.plot(list(range(len(losses))),losses)
plt.xlabel('Epochs')
plt.ylabel('Loss')

In [ ]:
#vocab_size

In [ ]:
#np.array()

## Predict using Model

In [ ]:
# with tf.Session() as sess:
#     saver = tf.train.import_meta_graph('Basic_RNN_TF_API.ckpt.meta')
#     saver.restore(sess, './Basic_RNN_TF_API.ckpt')

#     X = sess.graph.get_tensor_by_name('Placeholder:0')
    
#     for _ in range(30): 
#         y_pred = sess.run('rnn/transpose:0', feed_dict={X: X_new})
#         #print(y_pred)
#         #print(y_pred.shape)
#         content += interpret(y_pred,vocabulary)
#         X_new = y_pred
        
# print(content)